In [4]:
from __future__ import print_function, division,unicode_literals
import os
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
import librosa

from io import open
import unicodedata
import string
import re
import random

from tqdm.auto import tqdm
import json

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
from torch import optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


import torch.nn.functional as Fi


import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T

print(torch.__version__)
print(torchaudio.__version__)


import string

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

1.9.0
0.9.0


In [10]:
table_trans = str.maketrans(dict.fromkeys(string.punctuation))  # OR {key: None for key in string.punctuation}


path ="/Users/dami.osoba/work/bawk/small_dataset/small/CV_unpacked/cv-corpus-6.1-2020-12-11/en/validated.tsv"
meta = pd.read_csv(path,sep="\t")
meta_path = meta.set_index('path')

def read_manifest(path):
    manifest = []
    with open(path, 'r') as f:
        for line in tqdm(f, desc="Reading manifest data"):
            line = line.replace("\n", "")
            data = json.loads(line)
            manifest.append(data)
    return manifest
train_manifest_path = '/Users/dami.osoba/work/bawk/small_dataset/commonvoice_train_manifest.json'
train_manifest_data = read_manifest(train_manifest_path)
# keep audio < 4s
train_text = [data['text'] for data in train_manifest_data if data['duration']<=4]
train_path = [data['audio_filepath'] for data in train_manifest_data if data['duration']<=4]
train_path_pd = pd.DataFrame(train_path,columns=['train_path'])

# remove unicode
sentences = [c.encode(encoding="ascii",errors="ignore").decode().translate(table_trans) for c in train_text]
char_dict = sorted(list(set([b for a in sentences for b in a])))
char_index = {a:char_dict.index(a) for a in char_dict}
dictOfindex = {char_dict.index(a):a for a in char_dict}
char_index['EOS'] = len(char_dict)+1

Reading manifest data: 0it [00:00, ?it/s]

In [234]:
train_path[0].replace('src/data','small_dataset')

'/Users/dami.osoba/work/bawk/small_dataset/small/train/wav/common_voice_en_203526.wav'

# Create voice dataset

In [18]:
class VoiceDataset(Dataset):
    def __init__(self, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
#         self.landmarks_frame = pd.read_csv(csv_file)
#         self.root_dir = root_dir
        self.transform = transform
        self.path_frame = train_path_pd

    def __len__(self):
        return len(self.path_frame)

    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        waveform, _ = torchaudio.load(self.path_frame.loc[idx][0].replace('src/data','small_dataset'),)
        label = self.path_frame.loc[idx][0].split("/")[-1].split("wav")[0]+"mp3"
        # transcription for audio
        trans = meta_path.loc[label]['sentence']
        # encode to ascii
        trans = trans.encode(encoding="ascii",errors="ignore").decode().translate(table_trans).lower()
        chars =[b for a in trans for b in a]
        coded = [char_dict.index(a) for a in chars]+[27]

        sample = {'waveform': waveform, 'transcription': coded,'sentence':trans}

        if self.transform:
            sample = self.transform(sample)

        return sample

# Create FFT transform

In [155]:
window_size = 25/1000
stride = 10/1000
sample_rate = 16000
n_fft =int(window_size *sample_rate)
win_length = None
hop_length = int(sample_rate*stride)
n_mels = 80
max_time = 4


mel_spectrogram = T.MelSpectrogram(
    sample_rate=sample_rate,
    n_fft=n_fft,
    hop_length=hop_length,
    center=True,
    pad_mode="reflect",
    power=2.0,
    norm='slaney',
    onesided=True,
    n_mels=n_mels,
    mel_scale="htk",
)

# melspec = mel_spectrogram(waveform)


class MelSpec(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """
    def __init__(self):
        
        self.window_size = 25/1000
        self.stride = 10/1000
        self.sample_rate = 16000
        self.n_fft =int(self.window_size *self.sample_rate)
        self.win_length = None
        self.hop_length = int(self.sample_rate*self.stride)
        self.n_mels = 80
        self.max_time = 4
        pass
#         assert isinstance(output_size, (int, tuple))
#         self.output_size = output_size

    def mel_spectrogram(self,a):
        mel_spec = T.MelSpectrogram(
                        sample_rate=self.sample_rate,
                        n_fft=self.n_fft,
                        hop_length=self.hop_length,
                        center=True,
                        pad_mode="reflect",
                        power=2.0,
                        norm='slaney',
                        onesided=True,
                        n_mels=self.n_mels,
                        mel_scale="htk")
        return mel_spec(a)
        

    def __call__(self, sample):
        waveform, transcription,sentence = sample['waveform'], sample['transcription'],sample['sentence']
        #zero pad waveform
        zero_pad = torch.zeros(1, sample_rate*max_time- waveform.size()[1])
        padding = torch.cat([waveform,zero_pad],1)
        # get spectrogram
        wave_spec = self.mel_spectrogram(padding)
        wave_spec = wave_spec.swapaxes(1,2)
        #change transcription list to tensor
        transcription = torch.tensor(transcription, dtype=torch.long, device=device)

        return {'waveform': wave_spec, 'transcription': transcription, 'sentence':sentence}
    
transformed_dataset = VoiceDataset(transform = MelSpec())

for i in range(len(transformed_dataset)):
    sample = transformed_dataset[i]

    print(i, sample['waveform'].size(), sample['transcription'],sample['sentence'])

    if i == 3:
        break

0 torch.Size([1, 401, 80]) tensor([20,  8,  1, 20,  0, 19, 15,  2,  5, 18,  5,  4,  0,  8,  9, 13,  0,  1,
         0, 12,  9, 20, 20, 12,  5, 27]) that sobered him a little
1 torch.Size([1, 401, 80]) tensor([15, 16,  5, 14,  0,  3, 15, 14,  6,  5, 19, 19,  9, 15, 14,  0,  9, 19,
         0,  7, 15, 15,  4,  0,  6, 15, 18,  0, 20,  8,  5,  0, 19, 15, 21, 12,
        27]) open confession is good for the soul
2 torch.Size([1, 401, 80]) tensor([ 2, 21, 20,  0, 20,  8,  5,  0,  5, 14,  7, 12,  9, 19,  8, 13,  1, 14,
         0, 23,  1, 19,  0,  5, 24, 21, 12, 20,  1, 14, 20, 27]) but the englishman was exultant
3 torch.Size([1, 401, 80]) tensor([ 9,  0,  1, 13,  0,  6, 15, 12, 12, 15, 23,  9, 14,  7,  0, 13, 25,  0,
         4,  5, 19, 20,  9, 14, 25, 27]) i am following my destiny


In [220]:
dictOfindex = { i : char_dict[i] for i in range(0, len(char_dict) ) }
dictOfchar = { char_dict[i]:i for i in range(0, len(char_dict) ) }

In [68]:
from torch.utils.data.dataloader import default_collate

In [122]:
def pad_sequence(batch):
    # Make all tensor in a batch the same length by padding with zeros
    batch = [item.t() for item in batch]    
    batch = torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=0.)
    return batch


def pad_collate(batch):
    max_input_len = float('-inf')
    max_target_len = float('-inf')

    for elem in batch:
        feature = elem['waveform']
        feature = feature.squeeze()
        trn = elem['transcription']
        max_input_len = max_input_len if max_input_len > feature.shape[0] else feature.shape[0]
        max_target_len = max_target_len if max_target_len > len(trn) else len(trn)

    for i, elem in enumerate(batch):
        f = elem['waveform']
        trn = elem['transcription']
        sentence = elem['sentence']
        f = f.squeeze()
        input_length = f.shape[0]
        input_dim = f.shape[1]
        print(max_input_len)
        # print('f.shape: ' + str(f.shape))
        feature = np.zeros((max_input_len, input_dim), dtype=np.float32)
        feature[:f.shape[0], :f.shape[1]] = f
        trn = np.pad(trn, (0, max_target_len - len(trn)), 'constant', constant_values=0)
        batch[i] = (feature, trn, input_length,sentence)
        # print('feature.shape: ' + str(feature.shape))
        # print('trn.shape: ' + str(trn.shape))

    batch.sort(key=lambda x: x[2], reverse=True)

    return default_collate(batch)



def collate_fn(batch):

    # A data tuple has the form:
    # waveform, sample_rate, label, speaker_id, utterance_number

    tensors, targets,sentence = [], [],[]

    # Gather in lists, and encode labels as indices
    for a in batch:
        tensors += [a['waveform']]
        targets += [a['transcription']]
        sentence += [a['sentence']]
                   
    # Group the list of tensors into a batched tensor
    tensors = tensors
#     targets = torch.stack(targets)
    targets = pad_sequence(targets)

    return tensors, targets,sentence


train_loader = DataLoader(transformed_dataset, batch_size=10,collate_fn=pad_collate,
                        shuffle=True, num_workers=0)

iterator = iter(train_loader)
x_batch,y,input_lengths,sentence = iterator.next()
print(x_batch,y,input_lengths,sentence)

399
399
399
399
399
399
399
399
399
399
tensor([[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.2996e-14, 4.5604e-14, 2.9653e-14,  ..., 3.3676e-14,
          3.3830e-14, 3.3594e-14],
         [1.4453e-10, 5.0718e-10, 8.4498e-10,  ..., 2.9822e-11,
          2.5825e-11, 2.4776e-11],
         ...,
         [3.3585e-08, 1.1786e-07, 9.8517e-08,  ..., 1.2780e-09,
          1.3241e-09, 6.2286e-10],
         [1.5616e-08, 5.4800e-08, 1.1578e-07,  ..., 2.9410e-09,
          1.0053e-09, 8.3122e-10],
         [5.8468e-09, 2.0518e-08, 3.2361e-08,  ..., 1.3928e-09,
          1.7140e-09, 9.3718e-10]],

        [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.000

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout=0.0, bidirectional=True):
        super(Encoder, self).__init__()
        self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout,
                          bidirectional=bidirectional)

    def forward(self, input_x, enc_len):
        total_length = input_x.size(1)  # get the max sequence length
        # print('total_length: ' + str(total_length))
        # print('input_x.size(): ' + str(input_x.size()))
        packed_input = pack_padded_sequence(input_x, enc_len, batch_first=True)
        # print('enc_len: ' + str(enc_len))
        packed_output, hidden = self.rnn(packed_input)
        output, _ = pad_packed_sequence(packed_output, batch_first=True, total_length=total_length)
        return output, hidden
    
hmm = nn.GRU(80,10,)
input_x = x_batch.size(1)
enc_len = x_batch.size(2)
total_length =x_batch.size(1)
packed_input = pack_padded_sequence(x_batch, input_lengths, batch_first=True)
hah, _= hmm(packed_input)
output, _ = pad_packed_sequence(hah, batch_first=True, total_length=total_length)


In [197]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

#         self.embedding = nn.Embedding(input_size, hidden_size,)
        self.embedding = nn.GRU(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden,total_length):
        output,_ = self.embedding(input)
        
        
        output, hidden = self.gru(output, hidden)
        m = nn.MaxPool1d(MAX_LENGTH)
        yy = m(hidden.swapaxes(1,2))
        yy = yy.swapaxes(1,2)
        return output, yy

    def initHidden(self):
        return torch.zeros(1, MAX_LENGTH, self.hidden_size, device=device)

In [157]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
#         output = self.embedding(input)
        print(output.shape)
        output = Fi.relu(output)
        print(hidden.shape,output.shape)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1,1, self.hidden_size, device=device)

In [207]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
#         embedded ,_= self.embedding(input)

        embedded = self.dropout(embedded)

        attn_weights = Fi.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = Fi.relu(output)
        output, hidden = self.gru(output, hidden)

        output = Fi.log_softmax(self.out(output[0]), dim=1)
        output_probs = torch.exp(output)
        return output, hidden, attn_weights,output_probs

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [209]:
teacher_forcing_ratio = 0.80

SOS_token = 28
EOS_token = 27


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion,total_length, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(1)
    tot_length = total_length
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

#     for ei in range(input_length):
#         encoder_output, encoder_hidden = encoder(
#             input_tensor[:,:,ei].view(1,1,80), encoder_hidden)
#         encoder_outputs[ei] = encoder_output[0, 0]

#     encoder_output, encoder_hidden = encoder(input_tensor.reshape(1,MAX_LENGTH,80), encoder_hidden)

    encoder_output, encoder_hidden = encoder(input_tensor, encoder_hidden,tot_length)

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention,output_probs = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention,output_probs = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            yay = torch.distributions.categorical.Categorical(output_probs)
            topi = yay.sample()
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [210]:
teacher_forcing_ratio = 1
MAX_LENGTH = 401
SOS_token = 28
EOS_token = 27


def train_dec(input_tensor, target_tensor, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(2)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

#     for ei in range(input_length):
#         encoder_output, encoder_hidden = encoder(
#             input_tensor[:,:,ei].view(1,1,80), encoder_hidden)
#         encoder_outputs[ei] = encoder_output[0, 0]

    encoder_output, encoder_hidden = encoder(input_tensor.reshape(1,MAX_LENGTH,80), encoder_hidden)

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_probs= decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden,decoder_probs = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            yay = torch.distributions.categorical.Categorical(decoder_probs)
            topi = yay.sample()
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [132]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [211]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)


def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    
    lns = len(transformed_dataset)

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_examples = np.random.choice(lns-1,n_iters)
                      
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters):
        training_pair = transformed_dataset[training_examples[iter-1]]
        input_tensor = training_pair['waveform']
        target_tensor = torch.tensor(training_pair['transcription'], dtype=torch.long, device=device).view(-1, 1) 
        tot = input_tensor.size(1)

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion,tot)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / (plot_every*1.0)
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)
    
    


In [215]:
trainIters(encoder1, attn_decoder1, 5000, print_every=200, plot_every=1000)

/Users/dami.osoba/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


0m 5s (- 2m 21s) (200 4%) 1.8154
0m 11s (- 2m 10s) (400 8%) 1.8000
0m 17s (- 2m 6s) (600 12%) 1.7867
0m 22s (- 1m 59s) (800 16%) 1.8090
0m 28s (- 1m 53s) (1000 20%) 1.8454
0m 34s (- 1m 47s) (1200 24%) 1.8118
0m 40s (- 1m 43s) (1400 28%) 1.8259
0m 46s (- 1m 38s) (1600 32%) 1.7823
0m 51s (- 1m 32s) (1800 36%) 1.8145
0m 57s (- 1m 25s) (2000 40%) 1.8017
1m 2s (- 1m 19s) (2200 44%) 1.8077
1m 7s (- 1m 13s) (2400 48%) 1.7915
1m 13s (- 1m 7s) (2600 52%) 1.8023
1m 18s (- 1m 1s) (2800 56%) 1.7559
1m 23s (- 0m 55s) (3000 60%) 1.7795
1m 27s (- 0m 49s) (3200 64%) 1.7794
1m 32s (- 0m 43s) (3400 68%) 1.7794
1m 36s (- 0m 37s) (3600 72%) 1.7110
1m 41s (- 0m 32s) (3800 76%) 1.7982
1m 46s (- 0m 26s) (4000 80%) 1.8029
1m 52s (- 0m 21s) (4200 84%) 1.7905
1m 57s (- 0m 16s) (4400 88%) 1.7212
2m 2s (- 0m 10s) (4600 92%) 1.7771
2m 6s (- 0m 5s) (4800 96%) 1.7364


# Train model

In [212]:
hidden_size = 30
encoder1 = EncoderRNN(80, hidden_size).to(device)
attn_decoder2 = DecoderRNN(hidden_size, 29).to(device)

attn_decoder1 = AttnDecoderRNN(hidden_size, 29, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 500, print_every=100, plot_every=200)

/Users/dami.osoba/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


0m 2s (- 0m 10s) (100 20%) 2.8779
0m 4s (- 0m 7s) (200 40%) 2.5967
0m 7s (- 0m 4s) (300 60%) 2.4585
0m 9s (- 0m 2s) (400 80%) 2.4133


In [228]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        choice = np.random.randint(200)
        print(choice)
        actual = transformed_dataset[choice]['sentence']
        ex = transformed_dataset[choice]['waveform']
        output_words, attentions,_ = evaluate(encoder, decoder, ex)
        output_sentence = ''.join(output_words)
        print(actual, '<', output_sentence)
        print('')

# Really bad model

In [229]:
evaluateRandomly(encoder1, attn_decoder1)


50
the current director is mohammad sarafraz < a king ant tosible on thi proning it wucks<EOS>

76
the children are not in school in june < is and print is is looks inturs<EOS>

168
you said it to me too < the link imads<EOS>

56
see you in the funny papers < what<EOS>

49
what time will you be heading across < i thinkt is to<EOS>

129
what do you expect from a pig but a grunt < i wold in<EOS>

39
yes < in<EOS>

95
nine < brengleviry perfippamad sain her pupsunimul for broim<EOS>

197
its not so bad < wickere in a fill hickake<EOS>

89
a half volley is a difficult shot to make < burl here now beini file for prrfall of the leccay<EOS>

